In [ ]:
# installing any packages (if required)
! pip install imutils

In [ ]:
# importing required libraries and dependencies
import numpy as np
import cv2
from imutils import paths
import matplotlib.pyplot as plt
import argparse
import os

import skimage
from skimage.transform import resize

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.layers import AveragePooling2D, Dropout,Flatten,Dense,Input
from keras.models import Model

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# fetching the dataset
dataset = "BT_Dataset/"
args={}
args["dataset"]=dataset

# sub-directories pointing to tumor and non-tumor MRI scans
y_images = os.listdir(dataset+"Tumor")
n_images = os.listdir(dataset+"NoTumor")

# splitting image paths, converting images into RGB channels, and resizing images
image_paths = list(paths.list_images(args["dataset"]))
data = []
labels = []
for path in image_paths:

    # path splitting
    label = path.split(os.path.sep)[-2]
    image = cv2.imread(path)

    # converting to RGB channels
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # resizing images to uniform size
    image = cv2.resize(image, (224, 224))

    # appending values to data and labels
    data.append(image)
    labels.append(label)
    
data = np.array(data) / 255.0
labels = np.array(labels)

# displaying the size of dataset
print(len(data))

In [ ]:
# function displaying sample set of processed MRI scans
def plotter(i):

    # MRI scan with tumors
    tumor = cv2.imread(dataset+"Tumor//"+y_images[i])
    tumor = skimage.transform.resize(tumor, (150, 150, 3) , mode = 'reflect')

    # MRI scan with no tumors
    no_tumor = cv2.imread(dataset+"NoTumor//"+n_images[i])
    no_tumor = skimage.transform.resize(no_tumor, (150, 150, 3))
    
    # concatenating for display
    pair = np.concatenate((tumor, no_tumor), axis=1)
    print("Tumor vs No Tumor MRI")
    plt.figure(figsize=(10,5))
    plt.imshow(pair)
    plt.show()

# plotting the images
for i in range(0,5):
    plotter(i)

In [ ]:
# setting up a label binarizer
LB = LabelBinarizer()  

#Initializing the label binarizer
labels = LB.fit_transform(labels)
labels = to_categorical(labels)

# train-test split
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

In [ ]:
# setting up base model and head models
base_model = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
head_model = base_model.output

# pooling
head_model = AveragePooling2D(pool_size=(4, 4))(head_model)

# flattening
head_model = Flatten(name="flatten")(head_model)

# creating connected layers using relu
head_model = Dense(64, activation="relu")(head_model)

# preventing overfitting
head_model = Dropout(0.5)(head_model)

# Dense using softmax activation
head_model = Dense(2, activation="softmax")(head_model)
model = Model(inputs=base_model.input, outputs=head_model)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# setting up learning rate, number of epochs and batch size
INIT_LR = 1e-3
EPOCHS = 10
BS = 8

# rotate images to create more data
augmented = ImageDataGenerator(
    rotation_range=15,
    fill_mode="nearest")

# set Adam optimizer
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)

# compiling the model
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

# training the model and storing loss records
R = model.fit_generator(
    augmented.flow(X_train, Y_train, batch_size=BS),
    steps_per_epoch=len(X_train) // BS,
    validation_data=(X_test, Y_test),
    validation_steps=len(X_test) // BS,
    epochs=EPOCHS)

In [ ]:
# plot the accuracy using the stored record R
plt.plot(R.history['accuracy'], label='train acc')
plt.plot(R.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('accuracy')

# plotting the loss using the stored record R
plt.plot(R.history['loss'], label='train loss')
plt.plot(R.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('loss')

In [ ]:
y_pred = model.predict(X_test, batch_size=BS)
y_pred = np.argmax(y_pred, axis=1)

# print accuracy score
print('Accuracy score : ',accuracy_score(Y_test.argmax(axis=1),y_pred)*100,'%')

# print classificaiton report
print(classification_report(Y_test.argmax(axis=1), y_pred,target_names=LB.classes_))

In [ ]:
# Plotting examples to check model
L, W = 6, 5
fig, axes = plt.subplots(L, W, figsize = (12, 12))
axes = axes.ravel()

# prediction using thr model on a random test set of size L*W
y_pred = model.predict(X_test, batch_size=BS)
for i in np.arange(0,L*W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction = {}\n True = {}'.format(y_pred.argmax(axis=1)[i], Y_test.argmax(axis=1)[i]))
    axes[i].axis('off')

# plotting the predictions
plt.subplots_adjust(wspace = 0.5, hspace=0.5)   

In [ ]:
# printing sensitivity & specificity using a confusion matrix
cmatrix = confusion_matrix(Y_test.argmax(axis=1), y_pred)
sensitivity = cmatrix [0, 0] / (cmatrix [0, 0] + cmatrix [0, 1])
specificity = cmatrix [1, 1] / (cmatrix [1, 0] + cmatrix [1, 1])
print('Confusion Matrix: ')
print(cmatrix)
print("sensitivity: {:.3f}".format(sensitivity))
print("specificity: {:.3f}".format(specificity))

In [22]:
# saving the model as .h5 file
model.save('Brain_model.h5')

In [ ]:
# import files to run the model on new inputs
import numpy as np
from flask import jsonify
from keras.preprocessing import image
from keras.models import load_model
from google.colab import files

In [ ]:
# running the model to detect tumor in an input MRI scan of suitable size
image1 = files.upload()
image_list = list(image1.keys())
image1 = image_list[0]
print('Image : ',image1, '\n')
print('File uploaded.\n')
new_model = load_model('Brain_model.h5')

# editing file to suit requirements
test_image = image.load_img(image1,target_size=(224,224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
print('generating results\n')

# predicting the result on the input scan
result = new_model.predict(test_image)
print('Results : ',result)
y_pred = np.argmax(result, axis=1)
print('Model prediction: ',result)
prediction = ""

# displaying the output of the model
if result[0][0]==1.0 and result[0][1]==0:
  prediction = "Error: check valid only for MRI scans"
elif y_pred == 1:
  prediction = 'Patient has Tumour'
else:
  prediction = 'Patient is Healthy'